In [ ]:
import typing

In [ ]:
import firedrake
import netgen.csg
import netgen.geom2d
import numpy as np
import ufl

In [ ]:
import viskex

Generate and plot meshes

In [ ]:
geo = netgen.geom2d.SplineGeometry()
geo.AddRectangle(p1=(-1.0, -1.0), p2=(1.0, 1.0), bc="rectangle", leftdomain=1, rightdomain=0)
geo.AddCircle(c=(0.0, 0.0), r=0.5, bc="circle", leftdomain=2, rightdomain=1)
geo.SetMaterial(1, "outer")
geo.SetMaterial(2, "inner")
geo.SetDomainMaxH(2, 0.02)
mesh_2d = firedrake.Mesh(geo.GenerateMesh(maxh=0.1))
del geo

In [ ]:
left = netgen.csg.Plane(netgen.csg.Pnt(0.0, 0.0, 0.0), netgen.csg.Vec(-1.0, 0.0, 0.0))
right = netgen.csg.Plane(netgen.csg.Pnt(1.0, 1.0, 1.0), netgen.csg.Vec(1.0, 0.0, 0.0))
front = netgen.csg.Plane(netgen.csg.Pnt(0.0, 0.0, 0.0), netgen.csg.Vec(0.0, -1.0, 0.0))
back = netgen.csg.Plane(netgen.csg.Pnt(1.0, 1.0, 1.0), netgen.csg.Vec(0.0, 1.0, 0.0))
bot = netgen.csg.Plane(netgen.csg.Pnt(0.0, 0.0, 0.0), netgen.csg.Vec(0.0, 0.0, -1.0))
top = netgen.csg.Plane(netgen.csg.Pnt(1.0, 1.0, 1.0), netgen.csg.Vec(0.0, 0.0, 1.0))
cube = left * right * front * back * bot * top
cube.bc("cube")
sphere = netgen.csg.Sphere(netgen.csg.Pnt(0.6, 0.6, 0.6), 0.5)
geo = netgen.csg.CSGeometry()
geo.Add(cube - sphere)
mesh_3d = firedrake.Mesh(geo.GenerateMesh(maxh=0.1))
del left, right, front, back, bot, top
del cube, sphere,
del geo

In [ ]:
viskex.firedrake.plot_mesh(mesh_2d)

In [ ]:
viskex.firedrake.plot_mesh(mesh_2d, dim=2)

In [ ]:
viskex.firedrake.plot_mesh(mesh_2d, dim=1)

In [ ]:
viskex.firedrake.plot_mesh(mesh_2d, dim=0)

In [ ]:
viskex.firedrake.plot_mesh(mesh_3d)

In [ ]:
viskex.firedrake.plot_mesh(mesh_3d, dim=3)

In [ ]:
viskex.firedrake.plot_mesh(mesh_3d, dim=2)

In [ ]:
viskex.firedrake.plot_mesh(mesh_3d, dim=1)

In [ ]:
viskex.firedrake.plot_mesh(mesh_3d, dim=0)

Plot subdomains

In [ ]:
viskex.firedrake.plot_mesh_entities(
    mesh_2d, 2, "subdomains", mesh_2d.cell_subset(1).indices
)

In [ ]:
viskex.firedrake.plot_mesh_entities(
    mesh_2d, 2, "subdomains", mesh_2d.cell_subset(1).indices,
    2 * np.ones_like(mesh_2d.cell_subset(1).indices)
)

In [ ]:
viskex.firedrake.plot_mesh_sets(mesh_2d, 2, "subdomains")

In [ ]:
viskex.firedrake.plot_mesh_sets(mesh_3d, 3, "subdomains")

Plot boundaries

In [ ]:
viskex.firedrake.plot_mesh_entities(
    mesh_2d, 1, "boundaries",
    mesh_2d.exterior_facets.measure_set("exterior_facet", 1).indices
)

In [ ]:
viskex.firedrake.plot_mesh_entities(
    mesh_2d, 1, "boundaries",
    mesh_2d.exterior_facets.measure_set("exterior_facet", "everywhere").size
    + np.hstack((
        mesh_2d.interior_facets.measure_set("interior_facet", 5).indices,
        mesh_2d.interior_facets.measure_set("interior_facet", 6).indices,
        mesh_2d.interior_facets.measure_set("interior_facet", 7).indices,
        mesh_2d.interior_facets.measure_set("interior_facet", 8).indices
    ))
)

In [ ]:
viskex.firedrake.plot_mesh_sets(mesh_2d, 1, "boundaries")

In [ ]:
viskex.firedrake.plot_mesh_sets(mesh_3d, 2, "boundaries")

Interpolate and plot scalar functions

In [ ]:
def prepare_scalar_field_cases(  # type: ignore[no-any-unimported]
    mesh: firedrake.Mesh,
    expression: typing.Callable[[ufl.core.expr.Expr], ufl.core.expr.Expr]
) -> typing.Tuple[firedrake.Function, typing.Tuple[ufl.core.expr.Expr, ufl.FunctionSpace]]:
    """Prepare scalar field cases."""
    scalar_function_space = firedrake.FunctionSpace(mesh, "CG", 2)
    scalar_field_ufl = expression(ufl.SpatialCoordinate(mesh))
    scalar_field = firedrake.interpolate(scalar_field_ufl, scalar_function_space)
    return scalar_field, (scalar_field_ufl, scalar_function_space)

In [ ]:
mesh_2d_scalar_field, mesh_2d_scalar_field_ufl = prepare_scalar_field_cases(
    mesh_2d, lambda x: x[0]**3 + x[1]**2)
mesh_3d_scalar_field, mesh_3d_scalar_field_ufl = prepare_scalar_field_cases(
    mesh_3d, lambda x: x[0]**3 + x[1]**2 + x[2]**4)

In [ ]:
viskex.firedrake.plot_scalar_field(mesh_2d_scalar_field, "scalar")

In [ ]:
viskex.firedrake.plot_scalar_field(mesh_2d_scalar_field, "scalar", warp_factor=1.0)

In [ ]:
viskex.firedrake.plot_scalar_field(mesh_2d_scalar_field_ufl, "scalar")

In [ ]:
viskex.firedrake.plot_scalar_field(mesh_3d_scalar_field, "scalar")

In [ ]:
viskex.firedrake.plot_scalar_field(mesh_3d_scalar_field_ufl, "scalar")

Interpolate and plot vector functions

In [ ]:
def prepare_vector_field_cases(  # type: ignore[no-any-unimported]
    mesh: firedrake.Mesh,
    expression: typing.Callable[[ufl.core.expr.Expr], ufl.core.expr.Expr]
) -> typing.Tuple[firedrake.Function, typing.Tuple[ufl.core.expr.Expr, ufl.FunctionSpace]]:
    """Prepare vector field cases."""
    vector_function_space = firedrake.VectorFunctionSpace(mesh, "CG", 2)
    vector_field_ufl = ufl.as_vector(expression(ufl.SpatialCoordinate(mesh)))
    vector_field = firedrake.interpolate(vector_field_ufl, vector_function_space)
    return vector_field, (vector_field_ufl, vector_function_space)

In [ ]:
mesh_2d_vector_field, mesh_2d_vector_field_ufl = prepare_vector_field_cases(
    mesh_2d, lambda x: (x[0]**3 + x[1]**2, x[0]**5 + x[1]**4))
mesh_3d_vector_field, mesh_3d_vector_field_ufl = prepare_vector_field_cases(
    mesh_3d, lambda x: (x[0]**3 + x[1]**2 + x[2]**4, x[0]**6 + x[1]**5 + x[2]**7, x[0]**9 + x[1]**8 + x[2]**10))

In [ ]:
viskex.firedrake.plot_vector_field(mesh_2d_vector_field, "vector")

In [ ]:
viskex.firedrake.plot_vector_field(mesh_2d_vector_field, "vector", glyph_factor=0.1)

In [ ]:
viskex.firedrake.plot_vector_field(mesh_2d_vector_field, "vector", warp_factor=1.0)

In [ ]:
viskex.firedrake.plot_vector_field(mesh_2d_vector_field_ufl, "vector")

In [ ]:
viskex.firedrake.plot_vector_field(mesh_3d_vector_field, "vector")

In [ ]:
viskex.firedrake.plot_vector_field(mesh_3d_vector_field, "vector", glyph_factor=0.1)

In [ ]:
viskex.firedrake.plot_vector_field(mesh_3d_vector_field, "vector", warp_factor=1.0)